<a href="https://colab.research.google.com/github/marketakvasova/LSEC_segmentation/blob/main/LSEC_fenestration_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automatic segmentation of electron microscope images**

This notebook is intended for using a neural network for the task of binary segmentation of fenestrations of Liver sinusoidal entdothelial cells (LSECs).

# How to use this notebook

This notebook works with data saved on your Google Drive. Network training requires pairs of images and their corresponding masks saved in two diferent folders. The image-mask pairs don't need to be named exactly the same, but they should correspond when sorted alphabetically.

In [ ]:
# @title  { display-mode: "form" }
#@markdown ##**Run this cell to connect to Google Drive**
#@markdown A new window will open where you will be able to connect.

#@markdown When you are connected, you can see your Drive content in the left sidebar under **Files**.

from google.colab import drive
drive.mount('/content/gdrive')

# 1. Setup

In [ ]:
# @title  { display-mode: "form" }
#@markdown ##**Import necessary libraries**
!python --version
!pip install wandb
!pip install torchmetrics
!pip install segmentation-models-pytorch

import segmentation_models_pytorch as smp
from torchmetrics.classification import Dice, BinaryJaccardIndex
import os
from google.colab import drive
import torch.cuda
from torch.utils.data import Dataset
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchvision import transforms
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim
from torchsummary import summary
import shutil
import cv2 as cv
from numpy.lib.stride_tricks import as_strided
import pywt
from scipy.stats import norm
from google.colab.patches import cv2_imshow
import gc
import wandb
from numba import njit
from scipy.signal import convolve2d
import math

# gc.collect()
drive.mount('/content/gdrive')
model_folder = "./gdrive/MyDrive/ROI_patches/my_model"
os.makedirs(model_folder, exist_ok=True)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # TODO: do not even try this, if the gpu is not connected
print(DEVICE)
biomodel_folder = os.path.join(model_folder, "bioimageio_model")
biomodel_path = os.path.join(biomodel_folder, "weights.pt")
os.makedirs(biomodel_folder, exist_ok=True)
LOAD_TRAINED_MODEL = False
model_path = os.path.join(model_folder,"my_checkpoint.pth.tar")

# Utils

In [ ]:
class MyDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transofrm=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transofrm
        self.images = sorted([f for f in os.listdir(self.image_dir) if os.path.isfile(os.path.join(self.image_dir, f))])
        self.masks = sorted([f for f in os.listdir(self.mask_dir) if os.path.isfile(os.path.join(self.mask_dir, f))])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.masks[index]) # mask and image need to be called the same
        image = cv.imread(img_path, cv.IMREAD_GRAYSCALE).astype(np.float32)
        mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE).astype(np.float32)
        # mask[mask == 255.0] = 1
        mask /= 255
        return image, mask

class TransformDataset(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        image, mask = self.dataset[index]
        augmentations = self.transform(image=image, mask=mask)
        image = augmentations["image"]
        mask = augmentations["mask"]
        return image, mask

    def __len__(self):
        return len(self.dataset)


def get_loaders(img_dir, mask_dir, split, batch_size, num_workers=4, pin_memory=True): # TODO: check these parameters
    data = MyDataset(
        image_dir=img_dir,
        mask_dir=mask_dir,
        transofrm=None
    )

    train_transform, val_transform = get_transforms()

    train_indices, test_indices = train_test_split(
        range(len(data)),
        test_size=split,
        random_state=42
    )
    train_data = TransformDataset(Subset(data, train_indices), train_transform)
    val_data = TransformDataset(Subset(data, test_indices), val_transform)

    train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True
    )

    val_loader = DataLoader(
        val_data,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False
    )

    return train_loader, val_loader, train_indices

def get_transforms():
    train_transform = A.Compose( # TODO: background(preprocessing?), intensity
        [
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            # A.Affine(shear=(0.5,1)),
            # A.Affine(scale=(-10, 10)),
            A.Normalize(
                mean = 0.0,
                std = 1.0,
                max_pixel_value=255.0, # normalization to [0, 1]
            ),
            ToTensorV2()
        ]
    )

    val_transform = A.Compose(
        [
            A.Normalize(
                mean = 0.0,
                std = 1.0,
                max_pixel_value=255.0,
            ),
            ToTensorV2()
        ]
    )
    return train_transform, val_transform

# test_transform = A.Compose(
#     [
#     A.Normalize(
#       mean = 0.0,
#       std = 1.0,
#       max_pixel_value=255.0,
#     ),
#         ToTensorV2()
#     ]
# )

test_transform = transforms.Compose([
    transforms.ToTensor(),
    # Add more transformations if needed
])


def merge_images(image, mask):
    merge = np.zeros((mask.shape[0], mask.shape[1], 3))
    merge[:, :, 0] = image # B channel (0, 1, 2) = (B, G, R)
    merge[:, :, 2] = image # R channel
    merge[:, :, 1] = mask # G channel
    merge[:, :, 2][mask == 255.0] = 255 # R channel
    merge = merge.astype('uint8')
    return merge


def merge_original_mask(image_path, mask_path, output_folder):
    image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
    merge = merge_images(image, mask)
    filename_ext = os.path.basename(image_path)
    filename, ext = os.path.splitext(filename_ext)
    cv.imwrite(os.path.join(output_folder, filename+"_original_mask_merge"+ext), merge)


def merge_masks(mask1_path, mask2_path, output_folder):
    mask1 = cv.imread(mask1_path, cv.IMREAD_GRAYSCALE)
    mask2 = cv.imread(mask2_path, cv.IMREAD_GRAYSCALE)
    # merge = merge_images(image, mask)
    merge = np.zeros((mask1.shape[0], mask1.shape[1], 3))

    merge[:, :, 1][mask1 == 255.0] = 255
    merge[:, :, 2][mask2 == 255.0] = 255

    filename_ext = os.path.basename(mask1_path)
    filename, ext = os.path.splitext(filename_ext)
    cv.imwrite(os.path.join(output_folder, filename+"_mask_compare"+ext), merge)


def create_weighting_patches(patch_size, edge_size):
    patch = np.ones((patch_size, patch_size), dtype=float)

    # Calculate the linear decrease values
    decrease_values = np.linspace(1, 0, num=edge_size)
    decrease_values = np.tile(decrease_values, (patch_size, 1))
    increase_values = np.linspace(0, 1, num=edge_size)
    increase_values = np.tile(increase_values, (patch_size, 1))

    # Middle patch
    # Apply linear decrease to all four edges
    middle = patch.copy()
    middle[:, 0:edge_size] *= increase_values
    middle[:, patch_size-edge_size:patch_size] *= decrease_values
    middle[0:edge_size, :] *= increase_values.T
    middle[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((middle*255).astype(np.uint8))

    # Left
    left = patch.copy()
    left[:, patch_size-edge_size:patch_size] *= decrease_values
    left[0:edge_size, :] *= increase_values.T
    left[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((left*255).astype(np.uint8))

    # Right
    right = patch.copy()
    right[:, 0:edge_size] *= increase_values
    right[0:edge_size, :] *= increase_values.T
    right[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((right*255).astype(np.uint8))

    # Top
    top = patch.copy()
    top[:, 0:edge_size] *= increase_values
    top[:, patch_size-edge_size:patch_size] *= decrease_values
    top[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((top*255).astype(np.uint8))

    # Bottom
    bottom = patch.copy()
    bottom[:, 0:edge_size] *= increase_values
    bottom[:, patch_size-edge_size:patch_size] *= decrease_values
    bottom[0:edge_size, :] *= increase_values.T
    # cv2_imshow((bottom*255).astype(np.uint8))

    # Left Top edge
    top_left = patch.copy()
    top_left[:, patch_size-edge_size:patch_size] *= decrease_values
    top_left[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((top_left*255).astype(np.uint8))

    # Right top edge
    top_right = patch.copy()
    top_right[:, 0:edge_size] *= increase_values
    top_right[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((top_right*255).astype(np.uint8))

    # Left bottom edge
    bottom_left = patch.copy()
    bottom_left[:, patch_size-edge_size:patch_size] *= decrease_values
    bottom_left[0:edge_size, :] *= increase_values.T
    # cv2_imshow((bottom_left*255).astype(np.uint8))

    # Right Bottom edge
    bottom_right = patch.copy()
    bottom_right[:, 0:edge_size] *= increase_values
    bottom_right[0:edge_size, :] *= increase_values.T
    # cv2_imshow((bottom_right*255).astype(np.uint8))

    return middle, top_left, top, top_right, right, bottom_right, bottom, bottom_left, left


def add_mirrored_border(image, border_size, window_size):
    height, width = image.shape

    bottom_edge = window_size - ((height + border_size) % (window_size - border_size))
    right_edge = window_size - ((width + border_size) % (window_size - border_size))

    top_border = np.flipud(image[0:border_size, :])
    bottom_border = np.flipud(image[height - border_size:height, :])
    bottom_zeros = np.zeros((bottom_edge-border_size, width), dtype = image.dtype)
    top_bottom_mirrored = np.vstack((top_border, image, bottom_border, bottom_zeros))

    left_border = np.fliplr(top_bottom_mirrored[:, 0:border_size])
    right_border = np.fliplr(top_bottom_mirrored[:, width - border_size:width])
    right_zeros = np.zeros((top_bottom_mirrored.shape[0], right_edge-border_size), dtype = image.dtype)
    mirrored_image = np.hstack((left_border, top_bottom_mirrored, right_border, right_zeros))
    return mirrored_image

def inference_on_image_with_overlap(model, image_path, output_folder):
    window_size = 512
    oh, ow = 50, 50
    # out_crop =
    input_image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    image_height, image_width = input_image.shape
    original_height, original_width = image_height, image_width

    # bottom_edge = (image_height + oh) % (window_size - oh)
    # right_edge = (image_height + ow) % (window_size - ow)

    mirrored_image = add_mirrored_border(input_image, oh, window_size)
    # print(mirrored_image.shape)
    image_height, image_width = mirrored_image.shape


    weights = np.zeros((image_height, image_width))
    # tryout = np.zeros((image_height, image_width))
    output_probs = np.zeros((image_height, image_width))
    output_mask = np.zeros((image_height, image_width))
    middle, top_left, top, top_right, right, bottom_right, bottom, bottom_left, left = create_weighting_patches(window_size, oh)

    for x in range(0, image_height-window_size+1, window_size - oh):
        for y in range(0, image_width-window_size+1, window_size - ow):
            # Choose weighting window
            # print(x, y)
            if x == 0:
                if y == 0:
                    # if original_height != window_size:
                    weighting_window = top_left
                    # print('top left')
                elif y == image_width - window_size:
                    # print('top right')
                    weighting_window = top_right
                else:
                    weighting_window = top
                    # print('top ')
            elif x == image_height - window_size:
                if y == 0:
                    weighting_window = bottom_left
                    # print('bottom left')
                elif y == image_width - window_size:
                    weighting_window = bottom_right
                    # print('bottom right')
                else:
                    weighting_window = bottom
                    # print('bottom')
            elif y == 0:
                weighting_window = left
                # print('left')
            elif y == image_width - window_size:
                weighting_window = right
                # print('right')
            else:
                weighting_window = middle
                # print('middle')
            square_section = mirrored_image[x:x + window_size, y:y + window_size]
            weights[x:x + window_size, y:y + window_size] += weighting_window
            # tryout[x:x + window_size, y:y + window_size] += np.ones((window_size, window_size))*weighting_window
            square_section = preprocess_image(square_section) # TODO: prehodit tohle, at se to dela jednou pro celej obrazek, ne pro patche?
            square_tensor = test_transform(square_section).unsqueeze(0).to(DEVICE)  # Add batch dimension

            # Forward pass through the model
            with torch.no_grad():
                output = torch.sigmoid(model(square_tensor)).float()

            # Scale the probablity to 0-255
            output = output*255
            # output = output.to(torch.uint8)
            output_pil = output.squeeze(0).cpu().numpy().squeeze()
            # cv2_imshow(output_pil)
            output_probs[x:x+window_size, y:y+window_size] += output_pil*weighting_window
    # Crop
    # cv.imwrite(os.path.join(output_folder, "probs"+".png"), output_probs)

    output_probs = output_probs[oh:original_height+oh, ow:original_width+ow]
    weights *= 255
    # weights = weights[:original_height, :original_width]*255
    # tryout = tryout[:original_height, :original_width]*255

    # Apply weights
    # output_probs /= weights

    # Create image from mask
    output_mask = np.where(output_probs > 127, 255, 0)
    output_mask = output_mask.astype(np.uint8)
    filename_ext = os.path.basename(image_path)
    filename, ext = os.path.splitext(filename_ext)
    # cv.imwrite(os.path.join(output_folder, filename+"_mirrored"+ext), mirrored_image)

    # Merge image with created mask
    out_mask_path = os.path.join(output_folder, filename+"_new_mask"+ext)
    merge = merge_images(input_image, output_mask)
    cv.imwrite(os.path.join(output_folder, filename+"_new_mask_merge"+ext), merge)

    # cv.imwrite(os.path.join(output_folder, filename+"_probs"+ext), output_probs)
    cv.imwrite(out_mask_path, output_mask)
    # cv.imwrite(os.path.join(output_folder, filename+"_weights"+ext), weights)
    return out_mask_path

# def inference_on_image_with_overlap(model, image_path, output_folder):
#     window_size = 512
#     oh, ow = 124, 124
#     input_image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
#     image_height, image_width = input_image.shape
#     original_height, original_width = image_height, image_width
#     bottom_edge = image_height % (window_size - oh)
#     right_edge = image_width % (window_size - ow)
#     mirrored_image = np.zeros((image_height+bottom_edge, image_width+right_edge)).astype(np.uint8)
#     mirrored_image[:image_height, :image_width] = input_image
#     mirrored_image[image_height:, :image_width] = np.flipud(input_image[image_height-bottom_edge:, :])
#     mirrored_image[:, image_width:] = np.fliplr(mirrored_image[:, image_width-right_edge:image_width])
#     image_height += bottom_edge
#     image_width += right_edge
#     weights = np.zeros((image_height, image_width))
#     # tryout = np.zeros((image_height, image_width))
#     output_probs = np.zeros((image_height, image_width))
#     output_mask = np.zeros((image_height, image_width))
#     middle, top_left, top, top_right, right, bottom_right, bottom, bottom_left, left = create_weighting_patches(window_size, oh)

#     for x in range(0, image_height-window_size+1, window_size - oh):
#         for y in range(0, image_width-window_size+1, window_size - ow):
#             # Choose weighting window
#             if x == 0:
#                 if y == 0:
#                     if original_height != window_size:
#                         weighting_window = top_left
#                     else:
#                         weighting_window = np.ones((window_size, window_size))
#                 elif y == window_size - ow - 1:
#                     weighting_window = top_right
#                 else:
#                     weighting_window = top
#             elif x == window_size - oh - 1:
#                 if y == 0:
#                     weighting_window = bottom_left
#                 elif y == window_size - ow - 1:
#                     weighting_window = bottom_right
#                 else:
#                     weighting_window = bottom
#             elif y == 0:
#                 weighting_window = left
#             elif y == window_size - ow - 1:
#                 weighting_window = right
#             else:
#                 weighting_window = middle
#             square_section = mirrored_image[x:x + window_size, y:y + window_size]
#             weights[x:x + window_size, y:y + window_size] = weighting_window
#             # tryout[x:x + window_size, y:y + window_size] += np.ones((window_size, window_size))*weighting_window
#             square_section = preprocess_image(square_section)
#             square_tensor = test_transform(square_section).unsqueeze(0).to(DEVICE)  # Add batch dimension

#             # Forward pass through the model
#             with torch.no_grad():
#                 output = torch.sigmoid(model(square_tensor)).float()

#             # Scale the probablity to 0-255
#             output = output*255
#             output = output.to(torch.uint8)
#             output_pil = output.squeeze(0).cpu().numpy()
#             output_probs[x:x+window_size, y:y+window_size] += output_pil.squeeze()*weighting_window
#     # Crop
#     output_probs = output_probs[:original_height, :original_width]
#     # weights = weights[:original_height, :original_width]*255
#     # tryout = tryout[:original_height, :original_width]*255

#     # Apply weights
#     # output_probs /= weights

#     # Create image from mask
#     output_mask = np.where(output_probs > 127, 255, 0)
#     output_mask = output_mask.astype(np.uint8)
#     filename_ext = os.path.basename(image_path)
#     filename, ext = os.path.splitext(filename_ext)

#     # Merge image with created mask
#     out_mask_path = os.path.join(output_folder, filename+"_mask"+ext)
#     merge = merge_images(input_image, output_mask)
#     cv.imwrite(os.path.join(output_folder, filename+"_merge"+ext), merge)

#     cv.imwrite(os.path.join(output_folder, filename+"_probs"+ext), output_probs)
#     cv.imwrite(out_mask_path, output_mask)
#     # cv.imwrite(os.path.join(output_folder, filename+"_weights"+ext), weights)
#     return out_mask_path


def preprocess_image(image):
    image = nlm_filt(image)
    # image = wavelet_denoise(image, threshold=1.5)
    # image = apply_clahe(image)
    # image = cv.medianBlur(image, 7)
    return image


def apply_clahe(image):
    clahe = cv.createCLAHE(clipLimit=0.8, tileGridSize=(8, 8))
    clahe_image = clahe.apply(image)
    return clahe_image


def create_image_patches(image_folder, mask_folder, output_folder, patch_size):
    image_patches_path = os.path.join(output_folder,'image_patches')
    mask_patches_path = os.path.join(output_folder,'mask_patches')
    # rejected_path = os.path.join(output_folder,'rejected')
    # print(image_path)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if os.path.exists(image_patches_path):
        shutil.rmtree(image_patches_path)
    os.mkdir(image_patches_path)
    if os.path.exists(mask_patches_path):
        shutil.rmtree(mask_patches_path)
    os.mkdir(mask_patches_path)
    # if os.path.exists(rejected_path):
    #     shutil.rmtree(rejected_path)
    # os.mkdir(rejected_path)

    patch_area = patch_size**2
    fenestration_area_thresh = 0.0 #0.01
    image_filenames = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
    image_filenames = sorted(image_filenames)
    mask_filenames = [f for f in os.listdir(mask_folder) if os.path.isfile(os.path.join(mask_folder, f))]
    mask_filenames = sorted(mask_filenames)

    for image_name, mask_name in zip(image_filenames, mask_filenames):
        # if image_name.endswith(".tif"): # TODO: tohle mozna odstranit
        input_path = os.path.join(image_folder, image_name)
        mask_path = os.path.join(mask_folder, mask_name)

        img = cv.imread(input_path, cv.IMREAD_GRAYSCALE)
        mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
        height, width = img.shape

        shape = (height // patch_size, width // patch_size, patch_size, patch_size)
        strides = (patch_size * width , patch_size , width, 1)
        # strides = (patch_size * width , patch_size)

        # img_strided = as_strided(img, shape=(width//patch_size, height//patch_size, patch_size, patch_size),
        #              strides=img.strides + img.strides, writeable=False)
        img_strided = as_strided(img, shape=shape,
                        strides=strides, writeable=False) #TODO: check if the patches do not overlap
        mask_strided = as_strided(mask, shape=shape,
                        strides=strides, writeable=False)

        for i in range(img_strided.shape[0]):
            for j in range(img_strided.shape[1]):
                img_patch = img_strided[i, j]
                mask_patch = mask_strided[i, j]
                # Compute the percentage of white pixels
                fenestration_area = np.sum(mask_patch == 255)
                # print(fenestration_area)
                # fenestration_percentage = fenestration_area/patch_area
                if fenestration_area >= fenestration_area_thresh:
                    patch_filename = f"{os.path.splitext(os.path.basename(image_name))[0]}_patch_{i}_{j}.tif"
                    # preprocess image
                    img_patch = preprocess_image(img_patch)
                    cv.imwrite(os.path.join(image_patches_path, patch_filename), img_patch)
                    cv.imwrite(os.path.join(mask_patches_path, patch_filename), mask_patch)
                    # print("written patch ", patch_filename)
                else:
                    print("not writing patch")
    return image_patches_path, mask_patches_path


# Denoising
#   References for non-local means filtering and noise variance estimation:
#
#   [1] Antoni Buades, Bartomeu Coll, and Jean-Michel Morel, A Non-Local
#       Algorithm for Image Denoising, Computer Vision and Pattern
#       Recognition 2005. CVPR 2005, Volume 2, (2005), pp. 60-65.
#   [2] John Immerkaer, Fast Noise Variance Estimation, Computer Vision and
#       Image Understanding, Volume 64, Issue 2, (1996), pp. 300-302

def estimate_degree_of_smoothing(I): # This is how the estimation is done in Matlab (see imnlmfilt in Matlab)
    H, W = I.shape
    I = I.astype(np.float32)
    kernel = np.array([[1, -2, 1], [-2, 4, -2], [1, -2, 1]])
    conv_result = np.abs(convolve2d(I[:, :], kernel, mode='valid'))
    res = np.sum(conv_result)
    degree_of_smoothing = (res * np.sqrt(0.5 * np.pi) / (6 * (W - 2) * (H - 2)))
    if degree_of_smoothing == 0:
        degree_of_smoothing = np.finfo(np.float32).eps
    return degree_of_smoothing


def nlm_filt(image):
    window_size = 5
    search_window_size = 21
    degree_of_smoothing = estimate_degree_of_smoothing(image)
    image = cv.fastNlMeansDenoising(image, None, h = degree_of_smoothing, templateWindowSize = 5, searchWindowSize = 21)
    return image


def anscombe_transform(data):
    return 2.0 * np.sqrt(data + 3.0/8.0)


def inverse_anscombe_transform(data):
    # Reference
    # https://github.com/broxtronix/pymultiscale/blob/master/pymultiscale/anscombe.py
    return (1.0/4.0 * np.power(data, 2) +
        1.0/4.0 * np.sqrt(3.0/2.0) * np.power(data, -1.0) -
        11.0/8.0 * np.power(data, -2.0) +
        5.0/8.0 * np.sqrt(3.0/2.0) * np.power(data, -3.0) - 1.0 / 8.0)


def wavelet_denoising(data, threshold=1.5, wavelet='coif4', threshold_type='soft'):
    coeffs = pywt.wavedec2(data, wavelet = wavelet, level=3)
    coeffs[-1] = tuple(pywt.threshold(c, threshold, threshold_type) for c in coeffs[-1])
    coeffs[-2] = tuple(pywt.threshold(c, threshold, threshold_type) for c in coeffs[-2])
    coeffs[-3] = tuple(pywt.threshold(c, threshold, threshold_type) for c in coeffs[-3])
    return pywt.waverec2(coeffs, wavelet)


def wavelet_denoise(image, threshold):
    image = anscombe_transform(image)
    image = wavelet_denoising(image, threshold)
    image = inverse_anscombe_transform(image)
    # TODO: not sure this is the correct way how to do this
    image = image/np.max(image)*255
    return image.astype(np.uint8)

## Data utils

##Training utils

In [ ]:
# This is the official implementation of BoundaryDOULoss https://arxiv.org/pdf/2308.00220.pdf
# Taken from: https://github.com/sunfan-bvb/BoundaryDoULoss/tree/main
class BoundaryDoULoss(nn.Module):
    def __init__(self, n_classes):
        super(BoundaryDoULoss, self).__init__()
        self.n_classes = n_classes

    def _one_hot_encoder(self, input_tensor):
        tensor_list = []
        for i in range(self.n_classes):
            temp_prob = input_tensor == i
            tensor_list.append(temp_prob.unsqueeze(1))
        output_tensor = torch.cat(tensor_list, dim=1)
        return output_tensor.float()

    def _adaptive_size(self, score, target):
        kernel = torch.Tensor([[0,1,0], [1,1,1], [0,1,0]])
        padding_out = torch.zeros((target.shape[0], target.shape[-2]+2, target.shape[-1]+2))
        padding_out[:, 1:-1, 1:-1] = target
        h, w = 3, 3

        Y = torch.zeros((padding_out.shape[0], padding_out.shape[1] - h + 1, padding_out.shape[2] - w + 1)).cuda()
        for i in range(Y.shape[0]):
            Y[i, :, :] = torch.conv2d(target[i].unsqueeze(0).unsqueeze(0), kernel.unsqueeze(0).unsqueeze(0).cuda(), padding=1)
        Y = Y * target
        Y[Y == 5] = 0
        C = torch.count_nonzero(Y)
        S = torch.count_nonzero(target)
        smooth = 1e-5
        alpha = 1 - (C + smooth) / (S + smooth)
        alpha = 2 * alpha - 1

        intersect = torch.sum(score * target)
        y_sum = torch.sum(target * target)
        z_sum = torch.sum(score * score)
        alpha = min(alpha, 0.8)  ## We recommend using a truncated alpha of 0.8, as using truncation gives better results on some datasets and has rarely effect on others.
        loss = (z_sum + y_sum - 2 * intersect + smooth) / (z_sum + y_sum - (1 + alpha) * intersect + smooth)

        return loss

    def forward(self, inputs, target):
        inputs = torch.softmax(inputs, dim=1)
        target = self._one_hot_encoder(target)

        assert inputs.size() == target.size(), 'predict {} & target {} shape do not match'.format(inputs.size(), target.size())

        loss = 0.0
        for i in range(0, self.n_classes):
            loss += self._adaptive_size(inputs[:, i], target[:, i])
        return loss / self.n_classes


def save_checkpoint(model, model_path):#, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    model.save(model_path)
    # torch.save(state, filename)

def save_state_dict(model, model_path):
    print("=> Saving checkpoint")
    torch.save(model.state_dict(), model_path)

def load_state_dict(model, model_path):
    print("=> Loading checkpoint")
    model.load_state_dict(torch.load(model_path))

def validate_model(model, loader, loss_fn):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()
    running_loss = 0
    with torch.no_grad():
        for idx, (x, y) in enumerate(loader):
            x = x.to(DEVICE)
            y = y.to(DEVICE).unsqueeze(1)
            # Forward
            preds = model(x)
            # loss_fn = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(16))
            # loss = loss_fn(preds, y)
            loss = get_loss(preds, y, loss_fn)
            running_loss += loss.cpu()
            preds = (preds > 0.5).float()

            # num_correct += (preds == y).sum()
            # num_pixels += torch.numel(preds)
            dice_score += (2*(preds*y).sum()) / (preds+y).sum() + 1e-8 # this is a better predictor
    # print(
    #     f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f} ()"
    # )
    dice_score = dice_score/(idx+1)
    val_loss = running_loss/(idx+1)
    # dice_score = dice_score/len(loader)
    # val_loss = running_loss/len(loader) #TODO: not sure this is correct(dividing by batch size?)
    # print(f"Dice score is {dice_score}")
    # val_losses.append(running_loss/len(loader))
    # dice_scores.append(dice_score.cpu())
    model.train()
    return val_loss, dice_score.cpu()



# def save_predictions_as_imgs(
#         loader, model, folder="saved_images", device="cpu"
# ):
#     model.eval()
#     for idx, (x, y) in enumerate(loader):
#         x = x.to(device=device)
#         with torch.no_grad():
#             preds = torch.sigmoid(model(x))
#             preds = (preds > 0.5).float()
#         # print(f"preds max{preds.max()}")
#         # print(f"y max {y.max()}")
#         # torchvision.utils.save_image(preds, os.path.join(folder, f"pred{idx}.png"))
#         # torchvision.utils.save_image(y.unsqueeze(1), os.path.join(folder, f"pred{idx}_correct.png"))
#             imshow(preds)
#             imshow(y.unsqueeze(1))
#         break # TODO: change this so it does not loop
#     model.train()
#     print("Saving prediction as images.")

def view_prediction(loader, model, device="cpu"):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            # output = torch.softmax(model(x), dim=1)
            output = torch.sigmoid(model(x))
            preds = (output > 0.5).float()
            preds = preds.cpu().data.numpy()
            output = output.cpu().data.numpy()
            for i in range(preds.shape[0]):
                f=plt.figure(figsize=(128,32))
                # Original image
                plt.subplot(1,5*preds.shape[0],i+1)
                x = x.cpu()
                plt.imshow(x[i, 0, :, :], cmap='gray') # preds is a batch
                plt.title('Validation image')
                # NN output(probability)
                plt.subplot(1,5*preds.shape[0],i+2)
                plt.imshow(output[i, 0, :, :], interpolation='nearest', cmap='magma') # preds is a batch
                plt.title('NN output')
                # Segmentation
                plt.subplot(1,5*preds.shape[0],i+3)
                plt.imshow(preds[i, 0, :, :], cmap='gray') # preds is a batch
                plt.title('Prediction')
                # True mask
                plt.subplot(1,5*preds.shape[0],i+4)
                plt.imshow(y.unsqueeze(1)[i, 0, :, :], cmap='gray')
                plt.title('Ground truth')
                # IoU
                plt.subplot(1,5*preds.shape[0],i+5)
                im1 = y.unsqueeze(1)[i, 0, :, :]
                im2 = preds[i, 0, :, :]
                plt.imshow(im1, alpha=0.8, cmap='Blues')
                plt.imshow(im2, alpha=0.6,cmap='Oranges')
                plt.title('IoU')

            plt.show()
            break # TODO: change this so it does not loop
    model.train()


# def getClassWeights(mask_path, train_indices):
#     mask_dir_list = sorted(os.listdir(mask_path))
#     class_count = np.zeros(2, dtype=int)
#     for i in train_indices:
#         mask = cv.imread(os.path.join(mask_path, mask_dir_list[i]), cv.IMREAD_GRAYSCALE) #np.array(Image.open(os.path.join(mask_path, mask_dir_list[i])).convert('L'), dtype=np.float32)
#         mask[mask == 255.0] = 1
#         class_count[0] += mask.shape[0]*mask.shape[1] - mask.sum()
#         class_count[1] += mask.sum()

#     n_samples = class_count.sum()
#     n_classes = 2

#     class_weights = n_samples / (n_classes * class_count)
#     return torch.from_numpy(class_weights)


## Debug

In [ ]:
from matplotlib.ticker import PercentFormatter

def show_fitted_ellipses(image_path, ellipses):
    image = cv.imread(image_path)
    for ellipse in ellipses:
        if ellipse is not None:
            cv.ellipse(image, ellipse, (0, 0, 255), 1)
            center, axes, angle = ellipse
            center_x, center_y = center
            major_axis_length, minor_axis_length = axes
            rotation_angle = angle
            # print(center_x, center_y)
            cv.circle(image, (int(center_x), int(center_y)),radius=1, color=(0, 0, 255), thickness=-1)

        # print("Center:", center)
        # print("Major Axis Length:", major_axis_length)
        # print("Minor Axis Length:", minor_axis_length)
        # print("Rotation Angle:", rotation_angle)

    cv2_imshow(image)

def fit_ellipses(filtered_contours, centers):
    ellipses = []
    for contour, cnt_center in zip(filtered_contours, centers):
        if len(contour) >= 5:  # Ellipse fitting requires at least 5 points
            ellipse = cv.fitEllipse(contour) # TODO: maybe try a different computation, if this does not work well on edges (probably ok)
            # ellipse = cv.minAreaRect(cnt) # the fitEllipse functions fails sometimes(when the fenestration is on the edge and only a part of it is visible)
            dist = cv.norm(cnt_center, ellipse[0])
            # print(dist)
            if dist < 20:
                ellipses.append(ellipse)
            else:
                ellipses.append(None)
        else:
            ellipses.append(None)
    return ellipses

def find_fenestration_contours(image_path):
    seg_mask = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    contours, _ = cv.findContours(seg_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    return contours
    # image = cv.cvtColor(seg_mask, cv.COLOR_GRAY2RGB)
    # image_el = image.copy()
    # cv.drawContours(image, contours, -1, (0, 0, 255), 1)
    # cv2_imshow(image)

    # Remove noise and small artifacts
    # min_contour_area = 10
    # filtered_contours = [cnt for cnt in contours if cv.contourArea(cnt) > min_contour_area]
    # return filtered_contours

def find_contour_centers(contours):
    contour_centers = []
    for cnt in contours:
        M = cv.moments(cnt)
        center_x = int(M['m10'] / (M['m00'] + 1e-10))
        center_y = int(M['m01'] / (M['m00'] + 1e-10))
        contour_centers.append((center_x, center_y))
    # print(contour_centers)
    return contour_centers

def equivalent_circle_diameter(major_axis_length, minor_axis_length):
    return math.sqrt(major_axis_length * minor_axis_length)


import seaborn as sns
import itertools



def show_statistics(fenestration_areas, fenestration_areas_from_ellipses, roundness_of_ellipses, equivalent_diameters, min_roundness=0, min_d=None, max_d=None):
    palette = itertools.cycle(sns.color_palette())
    plt.figure(figsize=(21, 5))

    # Plot histogram of fenestration areas
    plt.subplot(1, 4, 1)
    sns.histplot(fenestration_areas, stat='probability')
    # plt.hist(fenestration_areas, bins=20, color='red', edgecolor='black', density=density)
    plt.title('Histogram of Fenestration Areas')
    plt.xlabel('Area ($\mathrm{nm}^2$)')
    # plt.ylabel('Frequency')
    plt.grid(True)

    # Plot histogram of areas of fitted elipses
    plt.subplot(1, 4, 2)
    sns.histplot(fenestration_areas_from_ellipses, stat='probability', color=next(palette)) # this will be the first color (blue)
    # plt.hist(fenestration_areas_from_ellipses, bins=20, color='red', edgecolor='black', density=density)
    plt.title('Histogram of Fenestration Areas (fitted ellipses)')
    plt.xlabel('Area ($\mathrm{nm}^2$)')
    # plt.ylabel('Frequency')
    plt.grid(True)

    # Plot histogram of roundness
    plt.subplot(1, 4, 3)
    r = sns.histplot(roundness_of_ellipses, stat='probability', color=next(palette), binwidth=0.025)
    r.set(xlim=(min_roundness, None))
    # plt.hist(roundness_of_ellipses, bins=10, color='blue', edgecolor='black', density=density)
    plt.title('Histogram of Roundness')
    plt.xlabel('Roundness (-)')
    # plt.ylabel('Frequency')
    plt.grid(True)
    # print(np.array(roundness_of_ellipses).max())

    # Plot histogram of equivalent circle diameters
    plt.subplot(1, 4, 4)
    d = sns.histplot(equivalent_diameters, stat='probability', color=next(palette), binwidth=10)
    d.set(xlim=(0, max_d))
    # plt.hist(equivalent_diameters, bins=20, color='green', edgecolor='black', density=density)
    plt.title('Histogram of Equivalent Circle Diameters')
    plt.xlabel('Diameter (nm)')
    # plt.ylabel('Frequency')
    plt.grid(True)


    # plt.gca().yaxis.set_major_formatter(PercentFormatter(1))



# Mask statistics debug
# One pixel corresponds to 10.62 nm
image_path = "./gdrive/MyDrive/ROIs_manually_corrected/augment_mask/_0_379.tif"
image_path = "./gdrive/MyDrive/lsec_test/old11_CA150_NE_01_original_mask.tif" # Image from semiautomatic labeling


pixel_size_nm = 10.62
contours = find_fenestration_contours(image_path)
fenestration_areas = [cv.contourArea(cnt) * (pixel_size_nm**2) for cnt in contours]
contour_centers = find_contour_centers(contours)
ellipses = fit_ellipses(contours, contour_centers)

# Show image of fitted ellipses
# show_fitted_ellipses(image_path, ellipses)

roundness_of_ellipses = []
equivalent_diameters = []
fenestration_areas_from_ellipses = []

for ellipse in ellipses:
    center, axes, angle = ellipse
    # center_x, center_y = center
    major_axis_length, minor_axis_length = axes
    roundness = minor_axis_length/major_axis_length
    roundness_of_ellipses.append(roundness)
    # rotation_angle = angle
    diameter = pixel_size_nm * equivalent_circle_diameter(major_axis_length, minor_axis_length)
    equivalent_diameters.append(diameter)
    fenestration_areas_from_ellipses.append((diameter**2)/4*math.pi)

# show_statistics(fenestration_areas, fenestration_areas_from_ellipses, roundness_of_ellipses, equivalent_diameters)


# Display the number of circles and their fitted ellipses
print("Number of fenestrations:", len(contours))
print("Number of fitted ellipses:", len(ellipses))

# Insert input and output folders

In [ ]:
# @title  { display-mode: "form" }
#@markdown All Google Drive paths should start with ./gdrive/MyDrive/ (Check the folder structure in the left sidebar under **Files**).

#@markdown Insert folder containing LSEC images:
input_folder = './gdrive/MyDrive/' #@param {type:"string"}
#@markdown Insert where to save the output masks (the folder will be created if it does not exist yet):
output_folder = './gdrive/MyDrive/' #@param {type:"string"}

input_folder = input_folder.strip()
output_folder = output_folder.strip()

if not os.path.exists(input_folder):
    print("Input folder does not exist")
    exit()
if not os.path.exists(ground_truth_mask_folder):
    os.makedirs(output_folder)
    print(f'Created {output_folder}')

# Get model weights

In [ ]:
# @title  { display-mode: "form" }
#@markdown Insert model weights path:
model_path = './gdrive/MyDrive/' #@param {type:"string"}
model_path = model_path.strip()

# Segment images

In [ ]:
# @title  { display-mode: "form" }
#@markdown Run segmentation:
image_names = [f for f in sorted(os.listdir(input_folder)) if os.path.isfile(os.path.join(input_folder, f))]

model = build_model('vgg19+imagenet', 0.0, 'dice')
if torch.cuda.is_available():
    model.load_state_dict(torch.load(model_path))
else:
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

for image_name in image_names:
    print(image_name)
    image_path = os.path.join(input_folder, image_name)
    out_mask_path = inference_on_image_with_overlap(model, image_path, output_folder)
    print(f'Saving {out_mask_path}')

# **5. Exclude fenestrations based on diameter and roundness**

In [ ]:
# @title  { display-mode: "form" }
#@markdown ##**Insert the pixel size, and min and max fenestration diameters in nanometers:**

#@markdown All fenestration with a smaller or larger diameter than the chosen range will be removed from the crated masks.
#@markdown (Use dot '.' as the decimal separator, not comma ',').

#@markdown Roundness is computed as minor axis length/major axis length of a fitted ellipse.
pixel_size_nm = 10.62 #@param {type:"number"}
min_diameter_nm = 105 #@param {type:"number"}
max_diameter_nm = 500 #@param {type:"number"}
min_roundness = 0 # @param {type:"slider", min:0, max:1, step:0.1}
mask_path = './gdrive/MyDrive/lsecs/mask_edit_test' #@param {type:"string"}
#@markdown If this is checked, the old masks will be deleted.
rewrite_images = False # @param {type:"boolean"}
mask_path = mask_path.strip()
mask_names = sorted([f for f in os.listdir(mask_path) if os.path.isfile(os.path.join(mask_path, f))])

def remove_contour_from_mask(contour, mask):
    # Fill the contour with black pixels
    cv.drawContours(mask, [contour], -1, 0, thickness=cv.FILLED)
    return mask


def remove_fenestrations(mask_path, min_d, max_d, min_roundness, pixel_size_nm):
    contours = find_fenestration_contours(mask_path)
    fenestration_areas = [cv.contourArea(cnt) * (pixel_size_nm**2) for cnt in contours]
    contour_centers = find_contour_centers(contours)
    ellipses = fit_ellipses(contours, contour_centers)
    roundness_of_ellipses = []
    equivalent_diameters = []
    fenestration_areas_from_ellipses = []
    mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
    # cv2_imshow(mask)
    # show_fitted_ellipses(mask_path, ellipses)

    # Remove all contours that do not fit the chosen conditions
    # Also remove all contours that were too small to fit an ellipse
    for contour, ellipse in zip(contours, ellipses):
        if ellipse is not None:
            center, axes, angle = ellipse
            # center_x, center_y = center
            minor_axis_length, major_axis_length = axes
            # print(axes)
            roundness = minor_axis_length/major_axis_length
            if roundness >= min_roundness:
                roundness_of_ellipses.append(roundness)
            # rotation_angle = angle
            diameter = pixel_size_nm * equivalent_circle_diameter(major_axis_length, minor_axis_length)
            # print(contour)
            # print(diameter)
            if diameter < min_d or diameter > max_d or roundness < min_roundness or np.isnan(diameter):
                mask = remove_contour_from_mask(contour, mask)
            else:
                equivalent_diameters.append(diameter)
                fenestration_areas_from_ellipses.append((diameter**2)/4*math.pi)
        else:
            mask = remove_contour_from_mask(contour, mask)
    # cv2_imshow(mask)
    # show_statistics(fenestration_areas, fenestration_areas_from_ellipses, roundness_of_ellipses, equivalent_diameters, min_roundness, min_diameter_nm, max_diameter_nm)
    equivalent_diameters = np.array(equivalent_diameters)
    # print(equivalent_diameters)
    if len(equivalent_diameters) > 0:
        mean = int(np.nanmean(equivalent_diameters) + 0.5) # This is how to round numbers in python...
        std = int(np.nanstd(equivalent_diameters) + 0.5)
        print(f"Mean equavalent diameter: {mean} nm, std: {std} nm ")
    return mask


#TODO: ukazat statistiky pro celou slozku obrazku

if not rewrite_images:
    new_mask_path = os.path.join(mask_path, 'edited_masks')
    os.makedirs(new_mask_path, exist_ok=True)
else:
    new_mask_path = mask_path
# print(new_mask_path)
for mask_name in mask_names:
    # print(mask_name)
    mask_path_full = os.path.join(mask_path, mask_name)
    # print(mask_path)
    # mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
    edited_mask = remove_fenestrations(mask_path_full, min_diameter_nm, max_diameter_nm, min_roundness, pixel_size_nm)
    # print(os.path.join(new_mask_path, mask_name))
    cv.imwrite(os.path.join(new_mask_path, mask_name), edited_mask)
    # cv.imwrite(os.path.join(new_mask_path, mask_name), mask)




## Display the number of circles and their fitted ellipses
# print("Number of fenestrations:", len(contours))
# print("Number of fitted ellipses:", len(ellipses))


# Show mask statistics